# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,531.570673,0.949168,1234,532.519841,530.621505,1065.039681,1061.243010
6,953.523262,0.156393,1234,953.679655,953.366869,1907.359311,1906.733737
7,3.942247,-0.499604,1234,3.442643,4.441851,6.885286,8.883702
8,607.588643,0.023157,1234,607.611799,607.565486,1215.223599,1215.130973
9,294.456672,-0.563504,1234,293.893168,295.020176,587.786336,590.040353
10,65.888444,0.695667,1234,66.584111,65.192777,133.168222,130.385554
11,985.170017,0.572615,1234,985.742632,984.597401,1971.485264,1969.194803
12,920.669370,0.204171,1234,920.873541,920.465199,1841.747082,1840.930397
13,56.518894,-0.553827,1234,55.965067,57.072720,111.930135,114.145441
14,445.056885,0.558037,1234,445.614922,444.498848,891.229845,888.997696


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-532.519841,530.621505
6,-953.679655,953.366869
7,-3.442643,4.441851
8,-607.611799,607.565486
9,-293.893168,295.020176
10,-66.584111,65.192777
11,-985.742632,984.597401
12,-920.873541,920.465199
13,-55.965067,57.072720
14,-445.614922,444.498848


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-532.519841,530.621505
6,-953.679655,953.366869
7,-3.442643,4.441851
8,-607.611799,607.565486
9,-293.893168,295.020176
10,-66.584111,65.192777
11,-985.742632,984.597401
12,-920.873541,920.465199
13,-55.965067,57.072720
14,-445.614922,444.498848


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-532.51984063,  530.62150514],
       [-953.67965533,  953.36686854],
       [  -3.44264315,    4.44185125],
       [-607.61179937,  607.56548635],
       [-293.89316805,  295.02017649],
       [ -66.58411123,   65.19277697],
       [-985.74263212,  984.59740127],
       [-920.87354099,  920.46519869],
       [ -55.96506746,   57.0727205 ],
       [-445.61492234,  444.49884802]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1063.039681,-1061.243010
6,1905.359311,-1906.733737
7,4.885286,-8.883702
8,1213.223599,-1215.130973
9,585.786336,-590.040353
10,131.168222,-130.385554
11,1969.485264,-1969.194803
12,1839.747082,-1840.930397
13,109.930135,-114.145441
14,889.229845,-888.997696


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
